In [1]:
# ================================================
# 1. Imports
# ================================================
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import optuna
import mlflow
import mlflow.xgboost   # integration for XGBoost models

/Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ================================================
# 2. Load processed dataset
# ================================================
df = pd.read_csv("/Users/riadanas/Desktop/housing regression MLE/data/processed/feature_engineered.csv")

target = "price"
X = df.drop(columns=[target])
y = df[target]

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("Train shape:", X_train.shape)
print("Validation shape:", X_val.shape)


Train shape: (636545, 40)
Validation shape: (159137, 40)


In [3]:
# ================================================
# 3. Define Optuna objective function with MLflow
# ================================================
def objective(trial):
    # Define hyperparameter search space
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 200, 1000),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "gamma": trial.suggest_float("gamma", 0.0, 5.0),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10.0, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10.0, log=True),
        "random_state": 42,
        "n_jobs": -1,
        "tree_method": "hist",
    }

    # Start an MLflow run for each Optuna trial
    with mlflow.start_run(nested=True):
        model = XGBRegressor(**params)
        model.fit(X_train, y_train)

        # Evaluate on validation set
        y_pred = model.predict(X_val)
        rmse = float(np.sqrt(mean_squared_error(y_val, y_pred)))
        mae = float(mean_absolute_error(y_val, y_pred))
        r2 = float(r2_score(y_val, y_pred))

        # Log hyperparameters and metrics to MLflow
        mlflow.log_params(params)
        mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})

    # Optuna will minimize RMSE
    return rmse


In [4]:
import xgboost as xgb
print(xgb.__version__)

3.0.4


In [5]:
import sys, xgboost as xgb
print(sys.executable)        # should point to .../.venv/bin/python
print(xgb.__version__)       # should print 3.0.4
print(xgb.__file__)          # should live under .../.venv/...

/Users/riadanas/Desktop/housing regression MLE/.venv/bin/python
3.0.4
/Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/xgboost/__init__.py


In [6]:
import mlflow

# Force MLflow to use the project root `mlruns/`
mlflow.set_tracking_uri("/Users/riadanas/Desktop/housing regression MLE/mlruns")

In [7]:
# ================================================
# 4. Run Optuna study with MLflow experiment
# ================================================
# Set experiment name (creates if not exists)
mlflow.set_experiment("xgboost_optuna_housing")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=15)

print("Best params:", study.best_trial.params)


2025/08/28 14:40:38 INFO mlflow.tracking.fluent: Experiment with name 'xgboost_optuna_housing' does not exist. Creating a new experiment.
[I 2025-08-28 14:40:38,866] A new study created in memory with name: no-name-e26f9678-73ed-4b7e-8888-5c70926ea28d
[I 2025-08-28 14:40:43,519] Trial 0 finished with value: 66487.89317464223 and parameters: {'n_estimators': 676, 'max_depth': 3, 'learning_rate': 0.03339766810381413, 'subsample': 0.7700772911140027, 'colsample_bytree': 0.5439845498792244, 'min_child_weight': 3, 'gamma': 0.6763301172324143, 'reg_alpha': 3.209479007176088e-08, 'reg_lambda': 0.05908151331846091}. Best is trial 0 with value: 66487.89317464223.
[I 2025-08-28 14:40:50,207] Trial 1 finished with value: 47229.76572771624 and parameters: {'n_estimators': 458, 'max_depth': 7, 'learning_rate': 0.013237137144772705, 'subsample': 0.5976690582905835, 'colsample_bytree': 0.5354236695225968, 'min_child_weight': 1, 'gamma': 3.7343728102443348, 'reg_alpha': 0.1378051674112472, 'reg_lambda

Best params: {'n_estimators': 976, 'max_depth': 10, 'learning_rate': 0.1310044074276173, 'subsample': 0.5214946511323292, 'colsample_bytree': 0.7993912362377076, 'min_child_weight': 7, 'gamma': 0.2394571426076625, 'reg_alpha': 1.1398496336490957e-05, 'reg_lambda': 1.8760583491336693}


In [8]:
# ================================================
# 5. Train final model with best params and log to MLflow
# ================================================
best_params = study.best_trial.params
best_model = XGBRegressor(**best_params)
best_model.fit(X_train, y_train)

y_pred = best_model.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)

print("Final tuned model performance:")
print("MAE:", mae)
print("RMSE:", rmse)
print("R²:", r2)

# Log best model and metrics to MLflow
with mlflow.start_run(run_name="best_xgboost_model"):
    mlflow.log_params(best_params)
    mlflow.log_metrics({"rmse": rmse, "mae": mae, "r2": r2})
    # Save model artifact to MLflow
    mlflow.xgboost.log_model(best_model, name="model")


Final tuned model performance:
MAE: 9563.481884820176
RMSE: 20582.64951237764
R²: 0.9964387589431593


/Users/riadanas/Desktop/housing regression MLE/.venv/lib/python3.11/site-packages/xgboost/sklearn.py:1028: UserWarning: [14:43:32] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
2025/08/28 14:43:35 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/08/28 14:43:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
